In [1]:
import requests
import time
import numpy as np
import pandas as pd

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

Using TensorFlow backend.


# Second environmement

## Get the data

In [2]:
#Global attributes 
user_id = 'IAHIZPIW80WPGDW7P7JE'
user_id = user_id
base_url = "http://35.180.254.42"
url_reset = base_url + "/reset"
url_predict = base_url + "/predict"
params = {"user_id" : user_id}

def load_data(url_reset,params) :
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    return data

data = load_data(url_reset,params)
data.keys()

dict_keys(['item_history', 'nb_items', 'nb_users', 'next_item', 'next_user', 'next_variables', 'rating_history', 'user_history', 'variables_history'])

In [3]:
user_history = data["user_history"]
item_history = data["item_history"]
rating_history = data["rating_history"]

train_ratings = pd.DataFrame({"user_id": user_history, "item_id": item_history, "rating": rating_history})

In [4]:
def mean_square_error(true,predicted):
    return (true - predicted) ** 2
def mean_absolute_error(true,predicted):
    return abs(true - predicted)

## 1) Model with MF using embeddings

In [5]:
def recommender_system(train_x,train_y,embedding_size,nb_users,nb_items) :
    
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')
    
    user_embedding = Embedding(input_dim=nb_users,output_dim=embedding_size,
                               input_length=1, name='user_embedding')(user_id_input) # input_dim is the size of vocabulary
    item_embedding = Embedding(input_dim=nb_items,output_dim=embedding_size,
                               input_length=1, name='item_embedding')(item_id_input)
    
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    y = Dot(axes=1)([user_vecs, item_vecs])

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    
    model.compile(optimizer='adam', loss='MSE')
    model.fit(train_x, train_y,batch_size=64, epochs=50, validation_split=0.1,shuffle=True)

    
    return model

In [7]:
data = load_data(url_reset,params)

nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_user = data["next_user"]
next_item = data["next_item"]


nb_samples = 100
train_x = [train_ratings["user_id"], train_ratings["item_id"]]     
train_y = train_ratings["rating"] 
results = dict()

for i in np.arange(20, 100, 10) : 
    model = recommender_system(train_x,train_y,i,nb_users,nb_items)
    sum1, sum2 = 0, 0

    for j in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([[next_user], [next_item]])
        prediction = params["predicted_score"]
        test_data = requests.get(url=url_predict, params=params).json()
        rating = test_data["rating"]
        next_user = test_data["next_user"]
        next_item = test_data["next_item"]
        
        sum1 += mean_square_error(rating,prediction)
        sum2 += mean_absolute_error(rating,prediction)
        
    mse, mae = sum1/nb_samples, sum2/nb_samples
    results[i] = [mse, mae]
print(results)

Train on 1800 samples, validate on 200 samples
Epoch 1/50
1800/1800 [==============================] - 0s 219us/step - loss: 10.1285 - val_loss: 9.7864
Epoch 2/50
1800/1800 [==============================] - 0s 28us/step - loss: 10.1144 - val_loss: 9.7852
Epoch 3/50
1800/1800 [==============================] - 0s 37us/step - loss: 10.0981 - val_loss: 9.7816
Epoch 4/50
1800/1800 [==============================] - 0s 43us/step - loss: 10.0738 - val_loss: 9.7726
Epoch 5/50
1800/1800 [==============================] - 0s 37us/step - loss: 10.0355 - val_loss: 9.7531
Epoch 6/50
1800/1800 [==============================] - 0s 43us/step - loss: 9.9759 - val_loss: 9.7161
Epoch 7/50
1800/1800 [==============================] - 0s 49us/step - loss: 9.8863 - val_loss: 9.6533
Epoch 8/50
1800/1800 [==============================] - 0s 43us/step - loss: 9.7566 - val_loss: 9.5563
Epoch 9/50
1800/1800 [==============================] - 0s 35us/step - loss: 9.5780 - val_loss: 9.4149
Epoch 10/50
1800/180

Epoch 29/50
1800/1800 [==============================] - 0s 86us/step - loss: 1.1214 - val_loss: 2.2860
Epoch 30/50
1800/1800 [==============================] - 0s 65us/step - loss: 0.9803 - val_loss: 2.1345
Epoch 31/50
1800/1800 [==============================] - 0s 36us/step - loss: 0.8608 - val_loss: 2.0072
Epoch 32/50
1800/1800 [==============================] - 0s 21us/step - loss: 0.7591 - val_loss: 1.8947
Epoch 33/50
1800/1800 [==============================] - 0s 22us/step - loss: 0.6733 - val_loss: 1.8023
Epoch 34/50
1800/1800 [==============================] - 0s 28us/step - loss: 0.6029 - val_loss: 1.7226
Epoch 35/50
1800/1800 [==============================] - 0s 24us/step - loss: 0.5445 - val_loss: 1.6519
Epoch 36/50
1800/1800 [==============================] - 0s 21us/step - loss: 0.4963 - val_loss: 1.5881
Epoch 37/50
1800/1800 [==============================] - 0s 31us/step - loss: 0.4552 - val_loss: 1.5428
Epoch 38/50
1800/1800 [==============================] - 0s 47us

Epoch 7/50
1800/1800 [==============================] - 0s 31us/step - loss: 9.6270 - val_loss: 9.5931
Epoch 8/50
1800/1800 [==============================] - 0s 28us/step - loss: 9.3730 - val_loss: 9.4387
Epoch 9/50
1800/1800 [==============================] - 0s 19us/step - loss: 9.0298 - val_loss: 9.2126
Epoch 10/50
1800/1800 [==============================] - 0s 22us/step - loss: 8.5902 - val_loss: 8.9055
Epoch 11/50
1800/1800 [==============================] - 0s 20us/step - loss: 8.0517 - val_loss: 8.5071
Epoch 12/50
1800/1800 [==============================] - 0s 39us/step - loss: 7.4287 - val_loss: 8.0411
Epoch 13/50
1800/1800 [==============================] - 0s 27us/step - loss: 6.7425 - val_loss: 7.5036
Epoch 14/50
1800/1800 [==============================] - 0s 46us/step - loss: 6.0112 - val_loss: 6.9168
Epoch 15/50
1800/1800 [==============================] - 0s 24us/step - loss: 5.2752 - val_loss: 6.3190
Epoch 16/50
1800/1800 [==============================] - 0s 25us/st

1800/1800 [==============================] - 0s 30us/step - loss: 0.2079 - val_loss: 1.4724
Epoch 36/50
1800/1800 [==============================] - 0s 48us/step - loss: 0.1928 - val_loss: 1.4627
Epoch 37/50
1800/1800 [==============================] - 0s 49us/step - loss: 0.1793 - val_loss: 1.4526
Epoch 38/50
1800/1800 [==============================] - 0s 40us/step - loss: 0.1670 - val_loss: 1.4486
Epoch 39/50
1800/1800 [==============================] - 0s 52us/step - loss: 0.1557 - val_loss: 1.4451
Epoch 40/50
1800/1800 [==============================] - 0s 42us/step - loss: 0.1448 - val_loss: 1.4465
Epoch 41/50
1800/1800 [==============================] - 0s 56us/step - loss: 0.1346 - val_loss: 1.4421
Epoch 42/50
1800/1800 [==============================] - 0s 32us/step - loss: 0.1254 - val_loss: 1.4440
Epoch 43/50
1800/1800 [==============================] - 0s 33us/step - loss: 0.1167 - val_loss: 1.4469
Epoch 44/50
1800/1800 [==============================] - 0s 30us/step - loss

1800/1800 [==============================] - 0s 19us/step - loss: 5.4018 - val_loss: 6.5637
Epoch 14/50
1800/1800 [==============================] - 0s 19us/step - loss: 4.5403 - val_loss: 5.8200
Epoch 15/50
1800/1800 [==============================] - 0s 19us/step - loss: 3.7179 - val_loss: 5.1344
Epoch 16/50
1800/1800 [==============================] - 0s 19us/step - loss: 2.9906 - val_loss: 4.4861
Epoch 17/50
1800/1800 [==============================] - 0s 26us/step - loss: 2.3611 - val_loss: 3.9102
Epoch 18/50
1800/1800 [==============================] - 0s 22us/step - loss: 1.8485 - val_loss: 3.4261
Epoch 19/50
1800/1800 [==============================] - 0s 21us/step - loss: 1.4428 - val_loss: 3.0251
Epoch 20/50
1800/1800 [==============================] - 0s 21us/step - loss: 1.1275 - val_loss: 2.6873
Epoch 21/50
1800/1800 [==============================] - 0s 20us/step - loss: 0.8886 - val_loss: 2.4313
Epoch 22/50
1800/1800 [==============================] - 0s 20us/step - loss

1800/1800 [==============================] - 0s 38us/step - loss: 0.0664 - val_loss: 1.3762
Epoch 42/50
1800/1800 [==============================] - 0s 31us/step - loss: 0.0600 - val_loss: 1.3774
Epoch 43/50
1800/1800 [==============================] - 0s 24us/step - loss: 0.0542 - val_loss: 1.3826
Epoch 44/50
1800/1800 [==============================] - 0s 26us/step - loss: 0.0488 - val_loss: 1.3901
Epoch 45/50
1800/1800 [==============================] - 0s 26us/step - loss: 0.0438 - val_loss: 1.3892
Epoch 46/50
1800/1800 [==============================] - 0s 23us/step - loss: 0.0393 - val_loss: 1.3959
Epoch 47/50
1800/1800 [==============================] - 0s 23us/step - loss: 0.0352 - val_loss: 1.3990
Epoch 48/50
1800/1800 [==============================] - 0s 28us/step - loss: 0.0315 - val_loss: 1.4016
Epoch 49/50
1800/1800 [==============================] - 0s 22us/step - loss: 0.0282 - val_loss: 1.4044
Epoch 50/50
1800/1800 [==============================] - 0s 23us/step - loss

KeyboardInterrupt: 

##  Model with MF using embeddings and non-linearities

In [24]:
def recommender_embedding_nonlinearities(train_x,train_y,embedding_size,nb_users,nb_items) : 
    
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')
    user_embedding = Embedding(input_dim=nb_users,output_dim=embedding_size,
                           input_length=1, name='user_embedding')(user_id_input)
    item_embedding = Embedding(input_dim=nb_items,output_dim=embedding_size,
                           input_length=1, name='item_embedding')(item_id_input)
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    
    input_vecs = Concatenate()([user_vecs, item_vecs])
    input_vecs = Dropout(0.2)(input_vecs)
    x = Dense(64, activation='relu')(input_vecs)
    y = Dense(1)(x)

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer='adam', loss='MSE')
    model.fit(train_x, train_y,batch_size=64, epochs=50, validation_split=0.1,shuffle=True)

    return model

In [25]:
data = load_data(url_reset,params)

nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_user = data["next_user"]
next_item = data["next_item"]


nb_samples = 100
train_x = [train_ratings["user_id"], train_ratings["item_id"]]     
train_y = train_ratings["rating"] 
results = dict()

for i in np.arange(20, 100, 10) : 
    model = recommender_embedding_nonlinearities(train_x,train_y,i,nb_users,nb_items)
    sum1, sum2 = 0, 0

    for j in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([[next_user], [next_item]])
        prediction = params["predicted_score"]
        test_data = requests.get(url=url_predict, params=params).json()
        rating = test_data["rating"]
        next_user = test_data["next_user"]
        next_item = test_data["next_item"]
        
        sum1 += mean_square_error(rating,prediction)
        sum2 += mean_absolute_error(rating,prediction)
        
    mse, mae = sum1/nb_samples, sum2/nb_samples
    results[i] = [mse, mae]
print(results)

Train on 1800 samples, validate on 200 samples
Epoch 1/50
1800/1800 [==============================] - 1s 521us/step - loss: 9.2091 - val_loss: 7.5844
Epoch 2/50
1800/1800 [==============================] - 0s 32us/step - loss: 6.7307 - val_loss: 4.3879
Epoch 3/50
1800/1800 [==============================] - 0s 47us/step - loss: 3.0305 - val_loss: 1.5763
Epoch 4/50
1800/1800 [==============================] - 0s 48us/step - loss: 1.2506 - val_loss: 1.0750
Epoch 5/50
1800/1800 [==============================] - 0s 41us/step - loss: 0.8208 - val_loss: 0.8801
Epoch 6/50
1800/1800 [==============================] - 0s 53us/step - loss: 0.6474 - val_loss: 0.7982
Epoch 7/50
1800/1800 [==============================] - 0s 37us/step - loss: 0.5779 - val_loss: 0.7549
Epoch 8/50
1800/1800 [==============================] - 0s 47us/step - loss: 0.5538 - val_loss: 0.7459
Epoch 9/50
1800/1800 [==============================] - 0s 52us/step - loss: 0.5344 - val_loss: 0.7395
Epoch 10/50
1800/1800 [==

1800/1800 [==============================] - 0s 37us/step - loss: 0.4529 - val_loss: 0.7052
Epoch 30/50
1800/1800 [==============================] - 0s 26us/step - loss: 0.4600 - val_loss: 0.6992
Epoch 31/50
1800/1800 [==============================] - 0s 29us/step - loss: 0.4446 - val_loss: 0.6909
Epoch 32/50
1800/1800 [==============================] - 0s 29us/step - loss: 0.4319 - val_loss: 0.6866
Epoch 33/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.4270 - val_loss: 0.6794
Epoch 34/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.4201 - val_loss: 0.6660
Epoch 35/50
1800/1800 [==============================] - 0s 28us/step - loss: 0.4181 - val_loss: 0.6655
Epoch 36/50
1800/1800 [==============================] - 0s 27us/step - loss: 0.4207 - val_loss: 0.6658
Epoch 37/50
1800/1800 [==============================] - 0s 29us/step - loss: 0.4100 - val_loss: 0.6688
Epoch 38/50
1800/1800 [==============================] - 0s 29us/step - loss

1800/1800 [==============================] - 0s 35us/step - loss: 0.3709 - val_loss: 0.6747
Epoch 36/50
1800/1800 [==============================] - 0s 35us/step - loss: 0.3697 - val_loss: 0.6785
Epoch 37/50
1800/1800 [==============================] - 0s 95us/step - loss: 0.3562 - val_loss: 0.7084
Epoch 38/50
1800/1800 [==============================] - 0s 41us/step - loss: 0.3543 - val_loss: 0.7086
Epoch 39/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.3448 - val_loss: 0.6830
Epoch 40/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.3502 - val_loss: 0.6897
Epoch 41/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.3506 - val_loss: 0.6869
Epoch 42/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.3294 - val_loss: 0.6785
Epoch 43/50
1800/1800 [==============================] - 0s 33us/step - loss: 0.3252 - val_loss: 0.6762
Epoch 44/50
1800/1800 [==============================] - 0s 34us/step - loss

1800/1800 [==============================] - 0s 83us/step - loss: 0.3452 - val_loss: 0.7243
Epoch 42/50
1800/1800 [==============================] - 0s 59us/step - loss: 0.3501 - val_loss: 0.6961
Epoch 43/50
1800/1800 [==============================] - 0s 76us/step - loss: 0.3453 - val_loss: 0.6991
Epoch 44/50
1800/1800 [==============================] - 0s 63us/step - loss: 0.3464 - val_loss: 0.6997
Epoch 45/50
1800/1800 [==============================] - 0s 75us/step - loss: 0.3406 - val_loss: 0.6947
Epoch 46/50
1800/1800 [==============================] - 0s 49us/step - loss: 0.3230 - val_loss: 0.7183
Epoch 47/50
1800/1800 [==============================] - 0s 65us/step - loss: 0.3272 - val_loss: 0.7030
Epoch 48/50
1800/1800 [==============================] - 0s 49us/step - loss: 0.3296 - val_loss: 0.7061
Epoch 49/50
1800/1800 [==============================] - 0s 38us/step - loss: 0.3281 - val_loss: 0.7075
Epoch 50/50
1800/1800 [==============================] - 0s 39us/step - loss

##  Avec une couche en plus

In [26]:
def recommender_embedding_dropout(train_x, train_y, embedding_size, nb_users, nb_items):

    user_id_input = Input(shape=[1], name='user')
    item_id_input = Input(shape=[1], name='item')
    user_embedding = Embedding(input_dim=nb_users, output_dim=embedding_size,
                               input_length=1, name='user_embedding')(user_id_input)
    item_embedding = Embedding(input_dim=nb_items , output_dim=embedding_size,
                               input_length=1, name='item_embedding')(item_id_input)
    
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    input_vecs = Concatenate()([user_vecs, item_vecs])
    x = Dense(64, activation='relu')(input_vecs)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    y = Dense(1)(x)

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer='adam', loss='MSE')
    model.fit(train_x, train_y, batch_size=64, epochs=50, validation_split=0.1,
              shuffle=True)

    return model

In [27]:
data = load_data(url_reset,params)

nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_user = data["next_user"]
next_item = data["next_item"]


nb_samples = 100
train_x = [train_ratings["user_id"], train_ratings["item_id"]]     
train_y = train_ratings["rating"] 
results = dict()

for i in np.arange(20, 100, 10) : 
    model = recommender_embedding_dropout(train_x,train_y,i,nb_users,nb_items)
    sum1, sum2 = 0, 0

    for j in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([[next_user], [next_item]])
        prediction = params["predicted_score"]
        test_data = requests.get(url=url_predict, params=params).json()
        rating = test_data["rating"]
        next_user = test_data["next_user"]
        next_item = test_data["next_item"]
        
        sum1 += mean_square_error(rating,prediction)
        sum2 += mean_absolute_error(rating,prediction)
        
    mse, mae = sum1/nb_samples, sum2/nb_samples
    results[i] = [mse, mae]
print(results)

Train on 1800 samples, validate on 200 samples
Epoch 1/50
1800/1800 [==============================] - 1s 701us/step - loss: 9.4390 - val_loss: 7.6324
Epoch 2/50
1800/1800 [==============================] - 0s 35us/step - loss: 6.3240 - val_loss: 3.1201
Epoch 3/50
1800/1800 [==============================] - 0s 34us/step - loss: 2.1072 - val_loss: 1.3121
Epoch 4/50
1800/1800 [==============================] - 0s 39us/step - loss: 1.1854 - val_loss: 1.0112
Epoch 5/50
1800/1800 [==============================] - 0s 31us/step - loss: 0.8674 - val_loss: 0.8511
Epoch 6/50
1800/1800 [==============================] - 0s 34us/step - loss: 0.7415 - val_loss: 0.8053
Epoch 7/50
1800/1800 [==============================] - 0s 34us/step - loss: 0.6930 - val_loss: 0.8224
Epoch 8/50
1800/1800 [==============================] - 0s 30us/step - loss: 0.6383 - val_loss: 0.8148
Epoch 9/50
1800/1800 [==============================] - 0s 40us/step - loss: 0.6386 - val_loss: 0.7877
Epoch 10/50
1800/1800 [==

1800/1800 [==============================] - 0s 41us/step - loss: 0.4633 - val_loss: 0.7129
Epoch 30/50
1800/1800 [==============================] - 0s 33us/step - loss: 0.4571 - val_loss: 0.7051
Epoch 31/50
1800/1800 [==============================] - 0s 33us/step - loss: 0.4498 - val_loss: 0.6853
Epoch 32/50
1800/1800 [==============================] - 0s 37us/step - loss: 0.4598 - val_loss: 0.6952
Epoch 33/50
1800/1800 [==============================] - 0s 35us/step - loss: 0.4657 - val_loss: 0.6854
Epoch 34/50
1800/1800 [==============================] - 0s 72us/step - loss: 0.4114 - val_loss: 0.7035
Epoch 35/50
1800/1800 [==============================] - 0s 71us/step - loss: 0.4105 - val_loss: 0.7058
Epoch 36/50
1800/1800 [==============================] - 0s 47us/step - loss: 0.4281 - val_loss: 0.6984
Epoch 37/50
1800/1800 [==============================] - 0s 89us/step - loss: 0.4240 - val_loss: 0.6894
Epoch 38/50
1800/1800 [==============================] - 0s 62us/step - loss

1800/1800 [==============================] - 0s 41us/step - loss: 0.6941 - val_loss: 0.7518
Epoch 8/50
1800/1800 [==============================] - 0s 45us/step - loss: 0.6572 - val_loss: 0.7554
Epoch 9/50
1800/1800 [==============================] - 0s 42us/step - loss: 0.6546 - val_loss: 0.7707
Epoch 10/50
1800/1800 [==============================] - 0s 70us/step - loss: 0.6574 - val_loss: 0.7913
Epoch 11/50
1800/1800 [==============================] - 0s 49us/step - loss: 0.6295 - val_loss: 0.7859
Epoch 12/50
1800/1800 [==============================] - 0s 37us/step - loss: 0.6237 - val_loss: 0.7474
Epoch 13/50
1800/1800 [==============================] - 0s 39us/step - loss: 0.6039 - val_loss: 0.7553
Epoch 14/50
1800/1800 [==============================] - 0s 43us/step - loss: 0.5775 - val_loss: 0.7589
Epoch 15/50
1800/1800 [==============================] - 0s 86us/step - loss: 0.5925 - val_loss: 0.7449
Epoch 16/50
1800/1800 [==============================] - 0s 41us/step - loss: 

1800/1800 [==============================] - 0s 79us/step - loss: 0.6136 - val_loss: 0.7587
Epoch 14/50
1800/1800 [==============================] - 0s 81us/step - loss: 0.6065 - val_loss: 0.7570
Epoch 15/50
1800/1800 [==============================] - 0s 125us/step - loss: 0.5990 - val_loss: 0.7463
Epoch 16/50
1800/1800 [==============================] - 0s 60us/step - loss: 0.5979 - val_loss: 0.7346
Epoch 17/50
1800/1800 [==============================] - 0s 49us/step - loss: 0.5486 - val_loss: 0.7341
Epoch 18/50
1800/1800 [==============================] - 0s 44us/step - loss: 0.5641 - val_loss: 0.7653
Epoch 19/50
1800/1800 [==============================] - 0s 40us/step - loss: 0.5144 - val_loss: 0.7534
Epoch 20/50
1800/1800 [==============================] - 0s 42us/step - loss: 0.5507 - val_loss: 0.7690
Epoch 21/50
1800/1800 [==============================] - 0s 39us/step - loss: 0.5196 - val_loss: 0.7484
Epoch 22/50
1800/1800 [==============================] - 0s 38us/step - los

## Avec les métadonnées

In [28]:
def recommender_system_covariates(train_x,train_y,embedding_size,nb_users,nb_items) : 

    user_id_input = Input(shape=[1], name='user')
    item_id_input = Input(shape=[1], name='item')
    meta_input = Input(shape=[5], name='meta_item')
    
    user_embedding = Embedding(input_dim=nb_users,output_dim=embedding_size,
                               input_length=1, name='user_embedding')(user_id_input)
    item_embedding = Embedding(input_dim=nb_items,output_dim=embedding_size,
                               input_length=1, name='item_embedding')(item_id_input)
    
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    input_vecs = Concatenate()([user_vecs, item_vecs, meta_input])
    
    x = Dense(64, activation='relu')(input_vecs)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    y = Dense(1)(x)

    model = Model(inputs=[user_id_input, item_id_input, meta_input], outputs=y)
    model.compile(optimizer='adam', loss='MSE')
    model.fit(train_x, train_y,
                batch_size=64, epochs=100, validation_split=0.1,
                shuffle=True)
    return model

In [29]:
data = load_data(url_reset,params)

nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_user = data["next_user"]
next_item = data["next_item"]

variables_history = np.array(data["variables_history"])
next_variable = np.array(data["next_variables"])

nb_samples = 100
train_x = [train_ratings["user_id"], train_ratings["item_id"], variables_history]     
train_y = train_ratings["rating"] 
results = dict()

for i in np.arange(50, 100, 10) : 
    model = recommender_system_covariates(train_x,train_y,i,nb_users,nb_items)
    sum1, sum2 = 0, 0

    for j in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([[next_user], [next_item],variables_history])
        prediction = params["predicted_score"]
        test_data = requests.get(url=url_predict, params=params).json()
        rating = test_data["rating"]
        next_user = test_data["next_user"]
        next_item = test_data["next_item"]
        next_variable = np.array(data["next_variables"])
        
        sum1 += mean_square_error(rating,prediction)
        sum2 += mean_absolute_error(rating,prediction)
        
    mse, mae = sum1/nb_samples, sum2/nb_samples
    results[i] = [mse, mae]
print(results)

Train on 1800 samples, validate on 200 samples
Epoch 1/100
1800/1800 [==============================] - 2s 1ms/step - loss: 8.8972 - val_loss: 5.0460
Epoch 2/100
1800/1800 [==============================] - 0s 42us/step - loss: 3.2829 - val_loss: 2.3363
Epoch 3/100
1800/1800 [==============================] - 0s 45us/step - loss: 2.0102 - val_loss: 1.6812
Epoch 4/100
1800/1800 [==============================] - 0s 44us/step - loss: 1.2894 - val_loss: 1.1714
Epoch 5/100
1800/1800 [==============================] - 0s 59us/step - loss: 0.7904 - val_loss: 0.9715
Epoch 6/100
1800/1800 [==============================] - 0s 46us/step - loss: 0.6383 - val_loss: 0.8678
Epoch 7/100
1800/1800 [==============================] - 0s 41us/step - loss: 0.6124 - val_loss: 0.8270
Epoch 8/100
1800/1800 [==============================] - 0s 43us/step - loss: 0.5877 - val_loss: 0.8213
Epoch 9/100
1800/1800 [==============================] - 0s 44us/step - loss: 0.5767 - val_loss: 0.8057
Epoch 10/100
1800/

Epoch 56/100
1800/1800 [==============================] - 0s 60us/step - loss: 0.1477 - val_loss: 0.9595
Epoch 57/100
1800/1800 [==============================] - 0s 43us/step - loss: 0.1508 - val_loss: 0.9627
Epoch 58/100
1800/1800 [==============================] - 0s 54us/step - loss: 0.1561 - val_loss: 0.9232
Epoch 59/100
1800/1800 [==============================] - 0s 51us/step - loss: 0.1602 - val_loss: 0.9353
Epoch 60/100
1800/1800 [==============================] - 0s 61us/step - loss: 0.1432 - val_loss: 0.9588
Epoch 61/100
1800/1800 [==============================] - 0s 60us/step - loss: 0.1340 - val_loss: 0.9689
Epoch 62/100
1800/1800 [==============================] - 0s 50us/step - loss: 0.1344 - val_loss: 0.9658
Epoch 63/100
1800/1800 [==============================] - 0s 52us/step - loss: 0.1378 - val_loss: 0.9742
Epoch 64/100
1800/1800 [==============================] - 0s 90us/step - loss: 0.1439 - val_loss: 0.9695
Epoch 65/100
1800/1800 [==============================]

1800/1800 [==============================] - 0s 47us/step - loss: 0.5377 - val_loss: 0.8023
Epoch 11/100
1800/1800 [==============================] - 0s 48us/step - loss: 0.5150 - val_loss: 0.8020
Epoch 12/100
1800/1800 [==============================] - 0s 50us/step - loss: 0.5023 - val_loss: 0.7926
Epoch 13/100
1800/1800 [==============================] - 0s 46us/step - loss: 0.4845 - val_loss: 0.7804
Epoch 14/100
1800/1800 [==============================] - 0s 47us/step - loss: 0.4618 - val_loss: 0.7767
Epoch 15/100
1800/1800 [==============================] - 0s 48us/step - loss: 0.4408 - val_loss: 0.7693
Epoch 16/100
1800/1800 [==============================] - 0s 46us/step - loss: 0.4325 - val_loss: 0.7986
Epoch 17/100
1800/1800 [==============================] - 0s 44us/step - loss: 0.4159 - val_loss: 0.7990
Epoch 18/100
1800/1800 [==============================] - 0s 54us/step - loss: 0.3962 - val_loss: 0.8038
Epoch 19/100
1800/1800 [==============================] - 0s 46us/st

1800/1800 [==============================] - 0s 55us/step - loss: 0.1024 - val_loss: 0.9011
Epoch 89/100
1800/1800 [==============================] - 0s 54us/step - loss: 0.0929 - val_loss: 0.9288
Epoch 90/100
1800/1800 [==============================] - 0s 54us/step - loss: 0.0868 - val_loss: 0.9195
Epoch 91/100
1800/1800 [==============================] - 0s 56us/step - loss: 0.0970 - val_loss: 0.9238
Epoch 92/100
1800/1800 [==============================] - 0s 50us/step - loss: 0.1001 - val_loss: 0.9394
Epoch 93/100
1800/1800 [==============================] - 0s 47us/step - loss: 0.0918 - val_loss: 0.9801
Epoch 94/100
1800/1800 [==============================] - 0s 51us/step - loss: 0.0921 - val_loss: 0.9586
Epoch 95/100
1800/1800 [==============================] - 0s 108us/step - loss: 0.0920 - val_loss: 0.9539
Epoch 96/100
1800/1800 [==============================] - 0s 110us/step - loss: 0.0975 - val_loss: 0.9593
Epoch 97/100
1800/1800 [==============================] - 0s 89us/

Epoch 66/100
1800/1800 [==============================] - 0s 48us/step - loss: 0.0999 - val_loss: 0.9710
Epoch 67/100
1800/1800 [==============================] - 0s 50us/step - loss: 0.1036 - val_loss: 0.9488
Epoch 68/100
1800/1800 [==============================] - 0s 52us/step - loss: 0.1090 - val_loss: 0.9699
Epoch 69/100
1800/1800 [==============================] - 0s 53us/step - loss: 0.1027 - val_loss: 0.9480
Epoch 70/100
1800/1800 [==============================] - 0s 56us/step - loss: 0.1016 - val_loss: 0.9568
Epoch 71/100
1800/1800 [==============================] - 0s 51us/step - loss: 0.1012 - val_loss: 0.9561
Epoch 72/100
1800/1800 [==============================] - 0s 49us/step - loss: 0.0974 - val_loss: 0.9456
Epoch 73/100
1800/1800 [==============================] - 0s 50us/step - loss: 0.0967 - val_loss: 0.9515
Epoch 74/100
1800/1800 [==============================] - 0s 54us/step - loss: 0.0968 - val_loss: 0.9803
Epoch 75/100
1800/1800 [==============================]